把题目给出中缀表达式转换为后缀表达式输出，并求后缀表达式的值。为简单起见，我们约定：1、输入的中缀表达式一定是合法的，并且只含数字，四种运算符+、-、*、/和小括号；2、运算数都是一位正整数(1～9)；3、输入的中缀表达式不超过20个字符；4、除法运算的结果仍然是正整数。

输入格式:
输入的第一行是一个正整数 N ，表示以下有 N 行。每行是一个中缀表达式。为简单起见，我们约定：1、输入的中缀表达式一定是合法的，并且只含数字，四种运算符+、-、*、/和小括号；2、运算数都是一位正整数(1～9)；3、输入的中缀表达式不超过20个字符；4、除法运算的结果仍然是正整数。

输出格式:
输出每行中缀表达式所对应后缀表达式，隔一个空格之后，输出该后缀表达式计算之后得到的值。

输入样例:
在这里给出一组输入。例如：
```
6
2+4
3+2*7
2*(4+6)
(5/2+4)*5+2
(3+5)*(7-2)/4
5*(8-(3+2))
```
输出样例:
在这里给出相应的输出。例如：
```
24+ 6
327*+ 17
246+* 20
52/4+5*2+ 32
35+72-*4/ 10
5832+-* 15
```

In [ ]:
import sys

# 读取全部输入并返回行迭代器
def read_lines():
    return iter(sys.stdin.read().splitlines())

# tokenization：把中缀表达式拆成数字、运算符和括号的 token 列表
# 支持负数（当 '-' 在行首或在 '(' / 运算符 后面时视为一元负号）
def tokenize(s: str):
    s = s.replace(' ', '')
    i = 0
    tokens = []
    prev_is_value = False  # 上一个 token 是否是值或右括号
    n = len(s)

    def read_number(start):
        j = start
        if j < n and s[j] == '-':  # 负号已判断为一元时包含进数字
            j += 1
        while j < n and s[j].isdigit():
            j += 1
        return s[start:j], j

    while i < n:
        c = s[i]
        if c.isdigit() or (c == '-' and not prev_is_value and i + 1 < n and s[i+1].isdigit()):
            num, i = read_number(i)
            tokens.append(num)
            prev_is_value = True
        elif c in '+*/^':
            tokens.append(c)
            i += 1
            prev_is_value = False
        elif c == '-':
            # 二元减号
            tokens.append('-')
            i += 1
            prev_is_value = False
        elif c == '(':
            tokens.append('(')
            i += 1
            prev_is_value = False
        elif c == ')':
            tokens.append(')')
            i += 1
            prev_is_value = True
        else:
            # 忽略其它字符（例如空白）
            i += 1
    return tokens

# 中缀转后缀（Shunting Yard），返回后缀 token 列表
def infix_to_postfix(tokens):
    prec = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3}
    right_assoc = {'^'}
    out = []
    st = []
    for t in tokens:
        # 数字直接入输出队列
        if t.lstrip('-').isdigit():
            out.append(t)
        elif t == '(':
            st.append(t)
        elif t == ')':
            # 弹出直到左括号
            while st and st[-1] != '(':
                out.append(st.pop())
            if not st:
                raise ValueError("mismatched paren")
            st.pop()
        else:
            # 运算符处理：根据优先级与结合性决定是否弹栈到输出
            while st and st[-1] != '(':
                top = st[-1]
                if top not in prec:
                    break
                if (t in right_assoc and prec[t] < prec[top]) or (t not in right_assoc and prec[t] <= prec[top]):
                    out.append(st.pop())
                else:
                    break
            st.append(t)
    while st:
        if st[-1] in '()':
            raise ValueError("mismatched paren")
        out.append(st.pop())
    return out

# 计算后缀表达式（整数运算，/ 向零截断）
def eval_postfix(tokens):
    st = []
    for t in tokens:
        if t.lstrip('-').isdigit():
            st.append(int(t))
        else:
            if len(st) < 2:
                raise ValueError("bad expr")
            b = st.pop()
            a = st.pop()
            if t == '+':
                st.append(a + b)
            elif t == '-':
                st.append(a - b)
            elif t == '*':
                st.append(a * b)
            elif t == '/':
                if b == 0:
                    raise ZeroDivisionError
                st.append(int(a / b))  # 向零截断
            elif t == '^':
                st.append(a ** b)
            else:
                raise ValueError("unknown op")
    if len(st) != 1:
        raise ValueError("bad expr")
    return st[0]

# 单行处理：中缀 -> 后缀字符串（无空格拼接） + 结果
def process_line(line):
    toks = tokenize(line)
    post = infix_to_postfix(toks)
    val = eval_postfix(post)
    return ''.join(post) + ' ' + str(val)

if __name__ == "__main__":
    it = read_lines()
    next(it, None)  # 跳过首行
    outs = []
    for line in it:
        line = line.strip()
        if not line:
            continue
        try:
            outs.append(process_line(line))
        except Exception:
            outs.append("ERROR")

    # 去掉末尾可能的空项并写出（最后一行不额外添加换行）
    while outs and outs[-1] == '':
        outs.pop()
    if outs:
        sys.stdout.write('\n'.join(outs))

In [3]:
def build_match_optimized(pattern):
    """
    优化的C风格BuildMatch，包含-1值
    更接近传统C语言的KMP实现
    """
    m = len(pattern)
    if m == 0:
        return []
    
    next_arr = [-1] * m  # 初始化为-1
    
    if m == 1:
        return next_arr
    
    next_arr[0] = -1
    next_arr[1] = 0
    
    i = 2  # 当前计算的位置
    j = 0  # 当前匹配的前缀长度
    
    while i < m:
        if pattern[i - 1] == pattern[j]:
            # 匹配成功，长度加1
            j += 1
            next_arr[i] = j
            i += 1
        elif j > 0:
            # 匹配失败，但j>0，回溯
            j = next_arr[j]
        else:
            # 匹配失败，且j=0
            next_arr[i] = 0
            i += 1
    
    return next_arr

def kmp_search_c_style(text, pattern):
    """
    C风格的KMP字符串匹配算法
    """
    n = len(text)
    m = len(pattern)
    
    if m == 0:
        return 0
    if n < m:
        return -1
    
    # 构建next数组
    next_arr = build_match_c_style(pattern)
    
    i = 0  # text指针
    j = 0  # pattern指针
    
    while i < n and j < m:
        if j == -1 or text[i] == pattern[j]:
            # 匹配成功或者j=-1（第一个字符就不匹配）
            i += 1
            j += 1
        else:
            # 匹配失败，根据next数组跳转
            j = next_arr[j]
    
    # 检查匹配结果
    if j == m:
        return i - j  # 返回匹配起始位置
    else:
        return -1  # 匹配失败

if __name__ == "__main__":
    arr = "ABABC"
    arr = build_match_optimized(list(arr))
    print(arr)

[-1, 0, 0, 1, 2]


In [4]:
import sys

lines = sys.stdin.read().splitlines()
it = iter(lines)

def getInput():
    try:
        return next(it).split()
    except StopIteration:
        return None

def StrToInt(arr):
    return [int(i) for i in arr]

def In():
    line = getInput()
    if line is None:
        return None
    return StrToInt(line)

if __name__ == "__main__":
    arr = In()  # 两个正整数，比如 4 3
    
    # 检查输入是否有效
    if arr is None or len(arr) < 2:
        print("输入错误：请提供两个正整数")
        exit(1)
    
    n, m = arr[0], arr[1]  # n个学生，数到m出列
    
    # 创建学生列表
    Stu = list(range(1, n + 1))
    
    # 约瑟夫环问题
    index = 0  # 当前报数位置
    while len(Stu) > 1:
        # 计算要删除的位置
        index = (index + m - 1) % len(Stu)
        # 移除该位置的学生
        Stu.pop(index)
    
    # 输出最后剩下的学生
    for ele in Stu:
        print(ele, end="")

输入错误：请提供两个正整数


TypeError: 'NoneType' object is not subscriptable